In [1]:
# set up matplotlib
%matplotlib widget
from matplotlib import pyplot as plt
plt.rcParams["figure.figsize"] = (8, 8)
plt.rcParams.update({"font.size": 14})

In [2]:
import numpy as np
import xarray as xr
from scipy.special import j0
from scipy.interpolate import RectBivariateSpline
from xbout import open_boutdataset

In [3]:
case = "1" # Case 1 is for anomalous density diffusion only.
filepath = "sq_mid3/BOUT.dmp.*.nc"
#filepath = "test" + str(case) + "/BOUT.dmp.0.nc"

ds = open_boutdataset(datapath=filepath, chunks={"t": 4})
dsn = xr.open_dataset("sq_mid_midres.fci.nc")

# Use squeeze() to get rid of the y-dimension, which has length 1 usually unless turbulent.
ds = ds.squeeze(drop=True)
dsn = dsn.squeeze(drop=True)

for key,val in ds.items():
    print(key)

Read in:
<xbout.BoutDataset>
Contains:
<xarray.Dataset> Size: 666MB
Dimensions:             (x: 260, y: 1, z: 260, t: 201)
Coordinates:
    dx                  (x, y, z) float64 541kB dask.array<chunksize=(260, 1, 260), meta=np.ndarray>
    dy                  (x, y, z) float64 541kB dask.array<chunksize=(260, 1, 260), meta=np.ndarray>
    dz                  (x, y, z) float64 541kB dask.array<chunksize=(260, 1, 260), meta=np.ndarray>
  * t                   (t) float64 2kB 0.0 5e+05 1e+06 ... 9.95e+07 1e+08
  * x                   (x) int64 2kB 0 1 2 3 4 5 6 ... 254 255 256 257 258 259
  * y                   (y) float64 8B 0.1963
  * z                   (z) float64 2kB 0.001923 0.005769 ... 0.9942 0.9981
Data variables: (12/52)
    Bxy                 (x, y, z) float64 541kB dask.array<chunksize=(260, 1, 260), meta=np.ndarray>
    G1                  (x, y, z) float64 541kB dask.array<chunksize=(260, 1, 260), meta=np.ndarray>
    G2                  (x, y, z) float64 541kB dask.array

/home/tirkas1/miniconda3/lib/python3.13/site-packages/xbout/load.py:341: UserWarning: No geometry type found, no physical coordinates will be added
  warn("No geometry type found, no physical coordinates will be added")


In [4]:
dx = ds["dx"].isel(z=0).values

# Get rid of existing "x" coordinate, which is just the index values.
#ds = ds.drop("x")

# Create a new coordinate, which is length in units of rho_s
ds = ds.assign_coords(x=np.arange(ds.sizes["x"])*dx)

#Store just initial values as well.
ds_initial = ds.isel(t=0)

In [5]:
dens = ds["Nh+"]
src = ds["SNh+"]
dens_true = dens.copy()

mask = dsn["in_mask"]
vols = dsn["vol_frac"]
ghost_id = dsn["ghost_id"].values
#Get mask of ghost and inside cells.
ghost_mask = (ghost_id >= 0).astype(bool)
full_mask = ghost_mask | mask.astype(bool)
#For cut cells, need to ignore ghost cells with vol 1 because BOUT runs all cells and 1 outside to not div by 0...
full_ghost_mask = ghost_mask & (vols >= 1).astype(bool)
full_mask = full_mask & ~full_ghost_mask
vols = np.where(full_mask, vols, 0)

gst_pts = dsn["ghost_pts"].values
bdy_pts = dsn["bndry_pts"].values
img_pts = dsn["image_pts"].values
normals = dsn["normals"].values
is_plasma = dsn["is_plasma"].values
Rarr = dsn["R"].values
Zarr = dsn["Z"].values

timestep_for_errs_early = 10
timestep_for_errs_late = 100
spl_e = RectBivariateSpline(Rarr[:,0], Zarr[0,:], dens.values[timestep_for_errs_early,:,:], kx=1, ky=1)
spl_l = RectBivariateSpline(Rarr[:,0], Zarr[0,:], dens.values[timestep_for_errs_late,:,:], kx=1, ky=1)

In [6]:
w_unit = 95788333.030660808    #Omega_ci
t_unit = 1/w_unit

nx = 256
nz = nx + 4
dx = np.float64(1.0) / np.float64(nx)
dz = np.float64(1.0) / np.float64(nz)

m = 1.0
n = 1.0
# Centers in x; faces/nodes in z if that's your convention
x = (np.arange(nx, dtype=np.float64) + 0.5) * dx     # centered x
z = (np.arange(nz, dtype=np.float64)) * dz           # unshifted z

ghosts_lo_x = x[0]  - dx*np.arange(2, 0, -1)
ghosts_hi_x = x[-1] + dx*np.arange(1, 2+1)
x = np.concatenate((ghosts_lo_x, x, ghosts_hi_x))
xx,zz = np.meshgrid(x,z,indexing='ij')

#Use physical grid values to map R,Z boundary to x,z (i.e. ignore ghosts)
R_min, R_max = dsn["R"].values[2,0], dsn["R"].values[-3,0]
Z_min, Z_max = dsn["Z"].values[0,0], dsn["Z"].values[0,-1]
dR = dsn["R"].values[1,0] - dsn["R"].values[0,0]
dZ = dsn["Z"].values[0,1] - dsn["Z"].values[0,0]
Lb, Rb = 1.7529296875, 3.2470703125
Bb, Tb = -0.7528846153846154, 0.7528846153846154
xL = x[2] + (Lb - R_min) * (x[-3] - x[2]) / (R_max - R_min)
xR = x[2] + (Rb - R_min) * (x[-3] - x[2]) / (R_max - R_min)
zB = z[0] + (Bb - Z_min) * (z[-1] - z[0]) / (Z_max - Z_min)
zT = z[0] + (Tb - Z_min) * (z[-1] - z[0]) / (Z_max - Z_min)

Lx = xR - xL
Lz = zT - zB
xi   = (x - xL)/ Lx
zeta = (z - zB)/ Lz
lbda = (m*np.pi/(Rb-Lb))**2 + (n*np.pi/(Tb-Bb))**2

xxi, zzeta = np.meshgrid(xi,zeta,indexing='ij',copy=False)
D      = 1
t64 = np.asarray(ds["t_array"].values, dtype=np.float64) # 1D time
t_norm = t64*t_unit
#spatial = np.cos(m*np.pi*xx)*np.cos(n*np.pi*zz)
#temp = np.exp(-lbda*D*(t_norm))
print(Lx)
print(Lz)
print(xL)
print(zB)

n0 = dens.values[0, 120,120]      # at t = 0
n1 = dens.values[1, 120,120]      # at t_index k (known step, but we ignore it for this calc)

ratio = (n1 - 1.0) / (n0 - 1.0)
t_eff = -np.log(ratio) / (0.2 * lbda)
print("Effective time from simulation at that point:", t_eff)
#t_norm = np.arange(201)*t_eff
# pick a grid point where the cosine mode is not zero
# the actual simulation timestep in physical units:
dt = t_norm[1] - t_norm[0]      # or however you compute Δt
ratio = (n1 - 1.0) / (n0 - 1.0)
lambda_eff = -np.log(ratio) / (D * dt)
print("Effective lambda from simulation at that point:", lambda_eff)
print("Analytic lambda:", lbda)
print("Percent error:", 100 * (lambda_eff - lbda)/lbda)

print("Simulation timestep output: ", t_norm[1])
spatial = np.cos(m * np.pi * xxi) * np.cos(n * np.pi * zzeta)
temp = np.exp(-D * lbda * t_norm)

#dens_true[:,:,:] = 1 + np.asarray(np.cos(np.pi*xxi)*np.cos(np.pi*zzeta))[np.newaxis,:,:]*np.asarray(temp)[:,np.newaxis,np.newaxis]
dens_true[:,:,:] = 1.0 + np.asarray(spatial)[np.newaxis,:,:]*np.asarray(temp)[:,np.newaxis,np.newaxis]


0.498046875
0.5019230769230768
0.2509765625
0.24903846153846154
Effective time from simulation at that point: 0.02609792055093576
Effective lambda from simulation at that point: 8.773470319371896
Analytic lambda: 8.773904434121475
Percent error: -0.004947794369525328
Simulation timestep output:  0.005219842377254392


In [7]:
M0 = (dens.where(mask)*vols*dR*dZ)[0,:,:].sum().values
M1 = (dens.where(mask)*vols*dR*dZ)[10,:,:].sum().values
M2 = (dens.where(mask)*vols*dR*dZ)[100,:,:].sum().values
M3 = (dens.where(mask)*vols*dR*dZ)[-1,:,:].sum().values
print(M0)
print(M1)
print(M2)
print(M3)
print("ΔM =", M3 - M0)
print("relative change =", (M3 - M0)/M0)
assert np.isclose(M3, M0, rtol=1e-10, atol=1e-14)

2.2498309795672977
2.2498309795672977
2.2498309795672986
2.2498309795672986
ΔM = 8.881784197001252e-16
relative change = 3.9477561993165617e-16


In [8]:
M0 = (dens_true.where(mask)*vols*dR*dZ)[0,:,:].sum().values
M1 = (dens_true.where(mask)*vols*dR*dZ)[10,:,:].sum().values
M2 = (dens_true.where(mask)*vols*dR*dZ)[100,:,:].sum().values
M3 = (dens_true.where(mask)*vols*dR*dZ)[-1,:,:].sum().values
print(M0)
print(M1)
print(M2)
print(M3)
print("ΔM =", M3 - M0)
print("relative change =", (M3 - M0)/M0)
assert np.isclose(M3, M0, rtol=1e-10, atol=1e-14)

2.2498309795672973
2.249830979567299
2.249830979567301
2.2498309795673013
ΔM = 3.9968028886505635e-15
relative change = 1.7764902896924533e-15


In [11]:

# Inputs assumed:
# bdy_pts: (Nb, 2) array, columns [R_b, Z_b] for boundary intercepts (BI)
# img_pts: (Nb, 2) array, columns [R_ip, Z_ip] for image points (inside fluid)
# dist_n:  (Nb,) array, distance along normal from BI to IP (physical coordinates)
# x_spl, z_spl: splines such that x_spl.ev(R,Z), z_spl.ev(R,Z) give computational coords
# x, z: 1D arrays of computational grid coords (same indexing as dens)
# dens: array with shape (nt, nx, nz) in (t,x,z) with indexing='ij'
# t_array: 1D array of physical times
# analytic_density(R, Z, t): function returning analytic density at given physical coords

def neumann_boundary_error_at_time(t_index):
    # --- 1. Physical boundary & image points ---
    Rb = bdy_pts[:, 0]
    Zb = bdy_pts[:, 1]
    Rip = img_pts[:, 0]
    Zip = img_pts[:, 1]

    # --- 2. Map to computational coordinates (x,z) ---
    spl_e = RectBivariateSpline(Rarr[:,0], Zarr[0,:], dens.values[k,:,:], kx=1, ky=1)
    n_num_b  = spl_e.ev(Rb,  Zb)
    n_num_ip  = spl_e.ev(Rip, Zip)

    # --- 4. Numerical normal derivative (one-sided) ---
    # du/dn ≈ (u(IP) - u(BI)) / dist_n
    du_dn_num = (n_num_ip - n_num_b) / np.hypot(normals[:,0], normals[:,1])

    # --- 5. Analytic normal derivative (same stencil) ---
    #t = t_array[t_index]

    #n_ana_b  = analytic_density(Rb,  Zb,  t)
    #n_ana_ip = analytic_density(Rip, Zip, t)

    spl_true_e = RectBivariateSpline(Rarr[:,0], Zarr[0,:], dens.values[k,:,:], kx=1, ky=1)
    n_num_true_b  = spl_true_e.ev(Rb,  Zb)
    n_num_true_ip  = spl_true_e.ev(Rip, Zip)
    du_dn_ana = (n_num_true_ip - n_num_true_b) / np.hypot(normals[:,0], normals[:,1])
    #du_dn_ana = 0 #(n_ana_ip - n_ana_b) / normals #TODO: Get BC condition?

    # If Neumann BC is zero-flux, du_dn_ana should be ~0; you can
    # just look at |du_dn_num|. Otherwise compare num vs ana:

    err = du_dn_num - du_dn_ana

    # L2 and Linf norms along the boundary
    L2_bnd   = np.sqrt(np.nanmean(err**2))
    Linf_bnd = np.nanmax(np.abs(err))

    # Relative norms (guard against zero analytic derivative)
    denom_L2   = max(np.nanmean(np.abs(du_dn_ana)), 1e-14)
    denom_Linf = max(np.nanmax(np.abs(du_dn_ana)), 1e-14)

    L2_rel   = L2_bnd   / denom_L2
    Linf_rel = Linf_bnd / denom_Linf

    return {
        "du_dn_num": du_dn_num,
        "du_dn_ana": du_dn_ana,
        "err": err,
        "L2_bnd": L2_bnd,
        "Linf_bnd": Linf_bnd,
        "L2_rel": L2_rel,
        "Linf_rel": Linf_rel,
    }

# Example usage:
k = 0  # some time index
res = neumann_boundary_error_at_time(k)
print("Boundary Neumann L2 error:", res["L2_bnd"])
print("Boundary Neumann L∞ error:", res["Linf_bnd"])


Boundary Neumann L2 error: 0.0
Boundary Neumann L∞ error: 0.0


In [ ]:
# Number of valid cells
N_valid = np.sum(mask)
#l2 err
dens_diff = np.abs(dens.where(mask)-dens_true.where(mask))
l2_err_t = np.sqrt(np.sum((dens_diff**2)*mask, axis=(1, 2)) / N_valid)

init_dens_diff = dens_diff[0,:,:]
mask2 = init_dens_diff.notnull() & (init_dens_diff >= 1e-10)
nerrs = int(mask2.sum().compute())

ii, jj = np.where(mask2)

if nerrs > 0:
    print(init_dens_diff.values[ii[0],jj[0]])
    print(dens.values[0,ii[0],jj[0]])
    print(dens_true.values[0,ii[0],jj[0]])

print(nerrs)

plt.close('all')      # optional, but guarantees a clean slate for the whole kernel
print(np.max(l2_err_t.values))
fig, ax = plt.subplots()  # new figure/axes each run
ax.plot(t_norm, l2_err_t)
ax.set_xlabel("Time [s]")
ax.set_ylabel("L2 error (RMS)")
ax.set_title("Time evolution of L2 error")
plt.show()


In [ ]:
dens_diff.where(mask).bout.animate2D(aspect='equal')

In [ ]:
#no_mask = ~(mask.astype(bool))
cut_mask = (vols < 1).astype(bool) & (vols > 0).astype(bool)
sim_mask = cut_mask.astype(bool) | mask.astype(bool)
src[:,2:-2,:].where(~sim_mask[2:-2,:]).bout.animate2D(aspect='equal')
#src.bout.animate2D(aspect='equal')